[View in Colaboratory](https://colab.research.google.com/github/paulanavarretec/Recsys-practicos/blob/master/Copia_de_Pra%CC%81ctico_pyreclab_3.ipynb)

# Ayudantía 3 - Sistemas Recomendadores: Pyreclab

** *texto en cursiva*Nombre(s):** Paula Navarrete - Astrid San Martín

## Setup

**Paso 1:** Descarga de archivos:

*   `dictionary.p`
*   `dictionary-stemm.p`
*  `tfidf_model.p`
*  `tfidf_model-stemm.p`

In [0]:
# Descargue los archivos ejecutando este comando
!curl -L -o 'resources.tar.gz' "https://drive.google.com/uc?export=download&id=1_Vp-veFfqCFkaEs-qVx99DYrAexBfq8w"

# Descomprima el archivo
!tar -xvf resources.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    388      0 --:--:--  0:00:01 --:--:--   318
100 1754k  100 1754k    0     0  1754k      0  0:00:01  0:00:01 --:--:-- 35.7M
resources/
resources/dictionary-stemm.p
resources/dictionary.p
resources/tfidf_model-stemm.p
resources/tfidf_model.p


**Paso 1.5:** Descarga del dataset:

In [0]:
# Puede descargar el dataset ejecutando el siguiente comando
!curl -L -o 'dataset.tar.gz' "https://drive.google.com/uc?export=download&id=1by4BZRPeUSnQRbwJWc-OKpIF6YBpCa7s"

# Y descomprimirlo con
!tar -xvf dataset.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    129      0 --:--:--  0:00:03 --:--:--   127
100 3117k    0 3117k    0     0  1039k      0 --:--:--  0:00:03 --:--:-- 1039k
./._corpus1.csv
corpus1.csv


**Paso 2:** Para este práctico es necesario instalar las siguentes dependencias:

In [0]:
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install pandas
!pip install numpy

  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
    100% |████████████████████████████████| 23.5MB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 12.9MB/s 
    100% |████████████████████████████████| 133kB 24.8MB/s 
    100% |████████████████████████████████| 4.6MB 5.6MB/s 
    100% |████████████████████████████████| 61kB 20.1MB/s 
    100% |████████████████████████████████| 552kB 17.7MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
import os
import nltk
import sklearn
import gensim
import string
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from gensim import corpora, models, similarities
from sklearn.neighbors import NearestNeighbors

## Preprocesamiento de datos

Lo primero es descargar las librerías de NLTK necesarias:

In [0]:
# Download corpora
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Para comenzar cargaremos el set de datos en un *dataframe* de Pandas, e imprimimos los 5 primeros registros para visualizar la estructura de los datos.

In [0]:
corpus_df = pd.read_csv('./corpus1.csv', sep='\t', header=None, encoding='latin')
corpus_df.columns = ['id', 'title', 'abstract']
corpus_df = corpus_df[['id', 'title', 'abstract']]
corpus_df[:5]

,id,title,abstract
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...


Lo siguiente es implementar una función que transforme texto no estructurado a una lista de tokens procesados.

In [0]:
stemm = False
stemmer = PorterStemmer()

def get_tokens(text):
    lowers = text.lower()
    no_punctuation = lowers.translate({ord(c): None for c in string.punctuation})
    tokens = nltk.word_tokenize(no_punctuation)
    if stemm:
        tokens = map(stemmer.stem, tokens)
        
    return tokens

get_tokens("I'm a super student for recommender systems!")

['im', 'a', 'super', 'student', 'for', 'recommender', 'systems']

**Pregunta:** Explique en sus palabras qué hace la función `get_tokens()`.

**Respuesta:**procesa el texto llevando todas las letras a minúsculas, sacando los signos de puntuación, y se crea un arreglo separando por palabras. No se retiran las "stop words". 



Ahora se tiene que generar un diccionario con todas las palabras del *corpus*.

Se recomienda revisar la documentación de gensim y leer cómo usar los diccionarios: [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [0]:
dict_file = './resources/dictionary-stemm.p' if stemm else './resources/dictionary.p'
if os.path.isfile(dict_file):
    dictionary = corpora.dictionary.Dictionary().load(dict_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dict_file)
    
corpus_df['tokenized_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df[:5]

,id,title,abstract,tokenized_abstract
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun..."


In [0]:
print(corpus_df['tokenized_abstract'][0][0],corpus_df['tokenized_abstract'][0][42])

we pca


**Pregunta:** Explique a qué corresponde la columna `tokenised_abstract` del dataframe.

**Respuesta:**La columna contiene el vector con las palabras después de pasar el abstract por la función *get_tokens*, es decir todas las letras en minúscula, sin puntación, palabras separadas de cada texto contenido en la columna abstract.


In [0]:
corpus_df['bow'] = corpus_df.tokenized_abstract.map(dictionary.doc2bow)
del corpus_df['tokenized_abstract']
corpus = corpus_df['bow'].tolist()
corpus_df[:5]

,id,title,abstract,bow
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22..."


**Pregunta:** Explique a qué corresponde la columna `bow`

**Respuesta:**Bag of Words (BOW) está representando el texto una vez pasado por la función *get_tokens*, en un vector de acuerdo al vocabulario. No interesa el orden de las palabras sólo la ocurrencia de la palabra. Es una forma de representar el texto para poder usarlo en un modelo. Se queda con las palabras presentes en el diccionario.



## Tf-idf

In [0]:
tfidf_model_file = 'resources/tfidf_model-stemm.p' if stemm else 'resources/tfidf_model.p'
if os.path.isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df[:5]

,id,title,abstract,bow,tf_idf
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(0, 0.19689725999527163), (1, 0.0861613877917..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35...","[(4, 0.0033554011043417254), (7, 0.02333778550..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27...","[(1, 0.06276351152911328), (4, 0.0049492930133..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17...","[(4, 0.0022699486545179476), (7, 0.01127724975..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22...","[(4, 0.001715799318906219), (5, 0.031751265629..."


**Pregunta:** Explicar a qué corresponde la columna `tf_idf` y por qué es útil en el procesamiento de texto. Mencione sus 2 principales partes, mediante la explicación del puntaje.

**Respuesta:** Se desea generar una puntuación para obtener la frecuencia de palabras para saber el contexto y poder determinar las palabras dominantes en el texto. Algunas palabras que no son de mucha utilidad van a aparecer con mucha frecuencia, como se quiere evitar eso. Es asi con TF-IDF se busca penalizar palabras que aparecen frecuentemente en un texto como "the" u otra palabra que entodos los textos aparecerá pero no da información real sobre el contexto del texto.
El proceso entonces entregar puntuación a la fecuencia de las palabras del documento (TF *term frecuency*), y luego generar putuación para las palabras "raras" ó poco frecuentes en todos los documentos (IDF *inverse document frecuency*).
Es útil en el procesamiento de texto porque se puede ponderar cuando una palabra frecuente es realmente un aporte para conocer el contexto ó es sólo una stopword (artículo, pronombre, etc.).


## Generar recomendaciones

En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se samplearán 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [0]:
# Random user

samples = corpus_df.sample(3)

for n, (ix, paper) in enumerate(samples.iterrows()):
  idx, title, abstract, bow, tf_idf = paper
  print('%d) %s' % (n+1, title))
  print('')
  print(abstract)
  print('\n' )

1) Domino Treewidth

We consider a special variant of tree-decompositions, called domino treedecompositions, and the related notion of domino treewidth. In a domino tree-decomposition, each vertex of the graph belongs to at most two nodes of the tree. We prove that for every k, d, there exists a constant c k;d such that a graph with treewidth at most k and maximum degree at most d has domino treewidth at most c k;d . The domino treewidth of a tree can be computed in O(n log n) time. There exist polynomial time algorithms that --- for fixed k --- decide whether a given graph G has domino treewidth at most k. If k is not fixed, this problem is NP-complete. The domino treewidth problem is hard for the complexity classes W [t] for all t 2 N, and hence the problem for fixed k is unlikely to be solvable in O(n ), where c is a constant, not depending on k.


2) Truffles - A Secure Service For Widespread File Sharing

Truffles is a system meant to address some of the major issues that still ma

In [0]:
# Recommendation functions

N = len(dictionary)

def to_sparse(matrix):
    return csr_matrix([gensim.matutils.sparse2full(row, length=N) for row in matrix]) 

def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(n_neighbors=(neighbors + 1), algorithm='brute', metric=metric).fit(X)
    return document_index

def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors([gensim.matutils.sparse2full(paper[model],length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i+=1
        print('\n')

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. Agregue nuevas celdas para cada implementación y/o pregunta.


** Pregunta:** Ejecute el modelo utilizando como representación tf-idf y una métrica de distancia euclideana. Modifique el parámetro nearest_neighbors a [5, 10, 20]. ¿qué efecto tiene el modelo en las recomendaciones observadas?

**Respuesta:**Las recomendaciones no cambian solo se van incorporando nuevas recomendaciones dependiendo si son 5, 10 ó 20 vecinos cercanos a entregar.

**Pregunta:** Eligiendo un valor fijo para *nearest neighbors* y utilizando representación tf-idf, ejecute el modelo con métrica de distancia *cosine*.¿Qué efecto tiene la métrica de distancia en las recomendaciones observadas?

**Respuesta:**La métrica *cosine* no generá ningún cambio en las recomendaciones con respecto a la distancia euclideana, al pedir los 10 vecinos cercanos estos son los mismos para las dos métricas.


In [0]:
# Recommendation example
 
doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf')

Domino Treewidth

Documentos cercanos: 
1. A Linear Time Algorithm for Finding Tree-Decompositions of Small Treewidth
2. Treewidth: Algorithmic techniques and results
3. Local Statistics For Random Domino Tilings Of The Aztec Diamond
4. A Tutorial on Particle Filters for On-line Nonlinear/Non-Gaussian Bayesian Tracking
5. Directed Tree-Width


Truffles - A Secure Service For Widespread File Sharing

Documentos cercanos: 
1. Truffles -- Secure File Sharing With Minimal System Administrator Intervention
2. Decentralized User Authentication in a Global File System
3. Usability and privacy: a study of Kazaa P2P file-sharing
4. Alex - a Global Filesystem
5. Intra-file Security for a Distributed File System


Position paper: Motivated Goal and Action Selection

Documentos cercanos: 
1. Motivation-Based Direction of Planning Attention in Agents With Goal Autonomy
2. Anytime Planning For Agent Behaviour
3. Induction in Noisy Domains
4. Agents and Petri Nets
5. Multi-level Control for Animated 

In [0]:
#Recomendacion tf-idf, distancia euclideana, nearest_neighbors 10
doc_idx1 = make_recommendations('tf_idf', 'euclidean', 10)
print_recommendations(doc_idx1, 'tf_idf')

Domino Treewidth

Documentos cercanos: 
1. A Linear Time Algorithm for Finding Tree-Decompositions of Small Treewidth
2. Treewidth: Algorithmic techniques and results
3. Local Statistics For Random Domino Tilings Of The Aztec Diamond
4. A Tutorial on Particle Filters for On-line Nonlinear/Non-Gaussian Bayesian Tracking
5. Directed Tree-Width
6. 1.5-Approximation for Treewidth of Graphs Excluding a Graph with One Crossing as a Minor
7. The Application of Classical Information Retrieval Techniques to Spoken Documents
8. What Cannot Be Computed Locally!
9. Constant-Time Distributed Dominating Set Approximation
10. Computational Scales Of Sobolev Norms With Application To Preconditioning


Truffles - A Secure Service For Widespread File Sharing

Documentos cercanos: 
1. Truffles -- Secure File Sharing With Minimal System Administrator Intervention
2. Decentralized User Authentication in a Global File System
3. Usability and privacy: a study of Kazaa P2P file-sharing
4. Alex - a Global File

In [0]:
#Recomendacion tf-idf, distancia euclideana, nearest_neighbors 20
doc_idx2 = make_recommendations('tf_idf', 'euclidean', 20)
print_recommendations(doc_idx2, 'tf_idf')

Domino Treewidth

Documentos cercanos: 
1. A Linear Time Algorithm for Finding Tree-Decompositions of Small Treewidth
2. Treewidth: Algorithmic techniques and results
3. Local Statistics For Random Domino Tilings Of The Aztec Diamond
4. A Tutorial on Particle Filters for On-line Nonlinear/Non-Gaussian Bayesian Tracking
5. Directed Tree-Width
6. 1.5-Approximation for Treewidth of Graphs Excluding a Graph with One Crossing as a Minor
7. The Application of Classical Information Retrieval Techniques to Spoken Documents
8. What Cannot Be Computed Locally!
9. Constant-Time Distributed Dominating Set Approximation
10. Computational Scales Of Sobolev Norms With Application To Preconditioning
11. Broadcast Encryption
12. On Geometric Optimization With Few Violated Constraints
13. Fixed-Parameter Tractability and Completeness I: Basic Results
14. Characteristic Varieties Of Arrangements
15. The Discrete Cosine Transform
16. The Union Of Convex Polyhedra In Three Dimensions
17. Arboricity and Bip

In [0]:
#Recomendacion tf-idf, distancia cosine, nearest_neighbors 10
doc_idx1 = make_recommendations('tf_idf', 'cosine', 10)
print_recommendations(doc_idx1, 'tf_idf')

Domino Treewidth

Documentos cercanos: 
1. A Linear Time Algorithm for Finding Tree-Decompositions of Small Treewidth
2. Treewidth: Algorithmic techniques and results
3. Local Statistics For Random Domino Tilings Of The Aztec Diamond
4. A Tutorial on Particle Filters for On-line Nonlinear/Non-Gaussian Bayesian Tracking
5. Directed Tree-Width
6. 1.5-Approximation for Treewidth of Graphs Excluding a Graph with One Crossing as a Minor
7. The Application of Classical Information Retrieval Techniques to Spoken Documents
8. What Cannot Be Computed Locally!
9. Constant-Time Distributed Dominating Set Approximation
10. Computational Scales Of Sobolev Norms With Application To Preconditioning


Truffles - A Secure Service For Widespread File Sharing

Documentos cercanos: 
1. Truffles -- Secure File Sharing With Minimal System Administrator Intervention
2. Decentralized User Authentication in a Global File System
3. Usability and privacy: a study of Kazaa P2P file-sharing
4. Alex - a Global File